In [84]:
from sklearn.neighbors import KNeighborsClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.model_selection import train_test_split as split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import RegressorChain
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import CategoricalNB
import sklearn.preprocessing as pre


import plotly.graph_objects as go
import numpy as np
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier


In [161]:
df = pd.read_csv('../data/clean_merged_data.csv')
df.set_index('index',inplace=True,drop=True)
df.sample()
df_dummy_col = pd.read_csv("../data/tracks_with_dummy_columns.csv")
df_dummy_col.sample()
multidf = pd.read_csv('../data/multi_output_format_tracks_categories.csv')
#multidf.set_index('index',inplace=True,drop=True)
multidf.sample()


,loudness,liveness,danceability,mode,popularity,key,energy,time_signature,valence,instrumentalness,speechiness,acousticness,tempo,track_name,track_id,category_name
39066,-4.098,0.857,0.705,1,64,1,0.56,4,0.502,0.0,0.036,0.699,105.027,Dois Enganados,7McCAKExE8KsI2q4Y8wx4U,Romance


In [86]:
#required features
features1 = ['loudness','liveness','danceability','mode','popularity','key','energy','time_signature','valence','instrumentalness','speechiness','acousticness','tempo','track_name','track_id','category_name']
sound_features = ['acousticness', 'danceability', 'energy', 'instrumentalness', 'liveness', 'valence','speechiness']
features2 = ['loudness', 'liveness', 'danceability', 'mode', 'popularity', 'key', 'energy', 'time_signature', 'valence', 'instrumentalness', 'speechiness', 'acousticness', 'tempo', 'track_name', 'track_id','category_name']
audio_features = ['loudness','liveness','danceability','mode','popularity','key','energy','time_signature','valence','instrumentalness','speechiness','acousticness','tempo']
categories_list =['Pop', 'Party', 'Dance/Electronic', 'Rock', 'R&B','At Home' ,'Romance', 'Chill', 'Jazz', 'Workout', 'Hip-Hop', 'Gaming', 'Summer', 'EQUAL', 'Indie', 'Mood','Travel', 'Indian Classical', 'Devotional', 'Wellness','RADAR', 'Classical','Instrumental',  'Sleep', 'Focus', 'Kids & Family','Cooking & Dining', 'Folk & Acoustic', 'Metal']

In [87]:
X = df_dummy_col[audio_features]
y = df_dummy_col[categories_list]
X_df_new_train, X_df_new_test, y_df_new_train, y_df_new_test = split(X, y, test_size=0.3, random_state=101)

In [88]:
X = df[audio_features]
y = df['category_name']
X_df_train, X_df_test, y_df_train, y_df_test = split(X, y, test_size=0.3, random_state=101)

In [89]:
#knn = KNeighborsClassifier()
knn = OneVsRestClassifier(KNeighborsClassifier()) 
knn.fit(X_df_new_train, y_df_new_train)
y_pred = knn.predict(X_df_new_test)

In [67]:
accuracy_score(y_pred,y_df_new_test)

0.14328166412336357

In [90]:
knn = KNeighborsClassifier()
knn.fit(X_df_new_train,y_df_new_train)

KNeighborsClassifier()

In [91]:
y_knn_predict = knn.predict(X_df_new_test)

In [92]:
accuracy_score(y_knn_predict,y_test)

0.14328166412336357

In [94]:
pipe_lr = Pipeline([('scaler', StandardScaler()),  ('knn_multi',MultiOutputClassifier(KNeighborsClassifier()))])

In [95]:
pipe_lr.fit(X_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('knn_multi',
                 MultiOutputClassifier(estimator=KNeighborsClassifier()))])

In [97]:
pipe_lr.score(X_test,y_test)


0.25371456107943136

In [98]:
clf = OneVsRestClassifier(knn)
clf.fit(X_df_new_train, y_df_new_train)

OneVsRestClassifier(estimator=KNeighborsClassifier())

In [99]:
y_ovs_knn_predicted = clf.predict(X_df_new_test)

In [100]:
accuracy_score(y_ovs_knn_predicted,y_df_new_test)

0.14328166412336357

In [101]:
gnb = MultiOutputClassifier(GaussianNB())
gnb.fit(X_df_new_train, y_df_new_train)

MultiOutputClassifier(estimator=GaussianNB())

In [102]:
accuracy_score(gnb.predict(X_df_new_test),y_df_new_test)

0.0363023050357401

In [103]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train,y_train)

DecisionTreeClassifier(random_state=0)

In [108]:
y_rpred = clf.predict(X_df_new_test)

In [109]:
accuracy_score(y_rpred,y_df_new_test)

0.2684121757288571

In [111]:
multi = MultiOutputClassifier(clf)
multi.fit(X_train,y_train)

MultiOutputClassifier(estimator=DecisionTreeClassifier(random_state=0))

In [112]:
y_pred = multi.predict(X_test)

In [113]:
accuracy_score(y_pred,y_test)

0.16175407597783312

In [115]:
clf1 = OneVsRestClassifier(clf)
clf1.fit(X_df_train, y_df_train)

OneVsRestClassifier(estimator=DecisionTreeClassifier(random_state=0))

In [116]:
y_pred_d = clf1.predict(X_df_test)

In [118]:
accuracy_score(y_pred_d,y_df_test)

0.1429356240323628

In [82]:
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import GaussianNB
from skmultilearn.problem_transform import LabelPowerset


In [122]:
# initialize classifier chains multi-label classifier
# with a gaussian naive bayes base classifier
classifier = ClassifierChain(GaussianNB())

# train
classifier.fit(X_df_new_train, y_df_new_train)

# predict
predictions = classifier.predict(X_df_new_test)

#accuracy_score(y_test,predictions)

In [127]:
gaussian = GaussianNB()
# train
gaussian.fit(X_df_train, y_df_train)

GaussianNB()

In [166]:
predictions = gaussian.predict(X_df_test)
accuracy_score(predictions,y_df_test)

0.2789791739499575